In [6]:
#ANNs
#protein yield prediction
#Import all the packages
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import average_precision_score
from sklearn.metrics import make_scorer
from sklearn import svm

from os import walk

In [19]:
def Neural_Network_Prediciton(df,output=False):
    #set the X(proteins) and y(soubility)
    X = df.iloc[:,0:-1]
    y = df.loc[:,'yield']
    
    # get binary data
    yc = y
    yc[y >= 0.2706] = True
    yc[y < 0.2706] = False

    X = X.as_matrix().astype(np.float)
    y = y.as_matrix().astype(np.float)
    yc = yc.as_matrix().astype(np.bool_)

    #set 10 fold validation
    kf = KFold(n_splits=10)

    def mc_reg(y_true, y_pred):
        y_true[y_true >= 0.2706] = True; y_true[y_true < 0.2706] = False
        y_pred[y_pred >= 0.2706] = True; y_pred[y_pred < 0.2706] = False
        return matthews_corrcoef(y_true, y_pred)
    
    def accracy_reg(y_true, y_pred):
        y_true[y_true >= 0.2706] = True; y_true[y_true < 0.2706] = False
        y_pred[y_pred >= 0.2706] = True; y_pred[y_pred < 0.2706] = False
        return accuracy_score(y_true, y_pred)
    
    def precision_reg(y_true,y_pred):
        y_true[y_true >= 0.2706] = True; y_true[y_true < 0.2706] = False
        y_pred[y_pred >= 0.2706] = True; y_pred[y_pred < 0.2706] = False
        return average_precision_score(y_true, y_pred)

    #use the best results for prediction
    clf = MLPClassifier(hidden_layer_sizes=(31, ),activation='relu',
                       learning_rate='invscaling',learning_rate_init=0.003)
    reg = MLPRegressor(hidden_layer_sizes=(31, ),activation='relu',
                       learning_rate='invscaling',learning_rate_init=0.003)
    
    #define score method
    method_binary = {'accuracy':'accuracy'}
    method_original = {'accuracy':make_scorer(accracy_reg),'precision':make_scorer(precision_reg),'AUC':'roc_auc','MCC':make_scorer(mc_reg),'r2':'r2'}
    
    results_binary = cross_validate(clf, X, yc, cv=kf, scoring=method_binary, return_train_score=False)
    results_original = cross_validate(reg, X, y, cv=kf, scoring=method_original, return_train_score=False)
    
    #out put the scores
    scores = pd.DataFrame([[np.mean(results_binary['test_accuracy']),np.mean(results_original['test_accuracy']),np.mean(results_original['test_precision']),
                                    np.mean(results_original['test_AUC']),np.mean(results_original['test_MCC']),
                                    np.mean(results_original['test_r2'])]],
                                    columns=['binary_accuracy','continuous_accuracy','precision','AUC','MCC','R2'])
    
    #output the predictions
    # create an empty dictionary for output
    predictions = {}
    if output == True:
        index = 0
        for train, test in kf.split(X):
            X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]
            reg.fit(X_train,y_train)
            y_predict = reg.predict(X_test)
            predictions[index] = pd.DataFrame(np.column_stack((y_predict,y_test)),columns=['Predicted','Original'])
            index+=1
        
    return scores, predictions

In [20]:
#first defind the headers of the columns (the different proteins) and then load data into dataframe
df_yield = pd.read_csv("Updated_data/cleanyieldN1.csv",index_col=0,header=0)
df_yield.dropna(axis=0,inplace=True)
yield_results, predictions = Neural_Network_Prediciton(df_yield)


In [23]:
writer = pd.ExcelWriter('yield_result.xlsx', engine='xlsxwriter')
yield_results.to_excel(writer,sheet_name='yield', startrow=0, startcol=0)

In [22]:
yield_results 

,binary_accuracy,continuous_accuracy,precision,AUC,MCC,R2
0,0.581033,0.516076,0.504154,0.605651,0.093656,0.01403
